<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/Analisis_hits_padel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inicialización de datos

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')


import os 
# path = '/content/gdrive/MyDrive/Colab Notebooks/TrackNetv2/3_in_3_out'
path = '/content/gdrive/MyDrive/PadelData/'
os.chdir(path)
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/PadelData


In [5]:
input_name = 'punto_qatar'#'FinalQatar_PremierPadel_Trim'#

In [6]:
import numpy as np
pista_campo_fondo = np.array([np.array([390,213]), np.array([890,213]),np.array([965,350]), np.array([315,350])])
pista_campo_cerca = np.array([[315,350], [965,350],[1114,622],[162,618]])
pista_aristas = np.array([[390,213], [890,213],[1114,622],[162,618]])
linea_saque_fondo = 250
linea_saque_cerca = 518
red_bajo = 350
red_alto = 300
pista_campo_fondo

array([[390, 213],
       [890, 213],
       [965, 350],
       [315, 350]])

In [7]:
import pickle
with open(input_name+'.playersbox', "rb") as fp:  
  person_results = pickle.load(fp)

import pickle
with open(input_name+'.ballpos', "rb") as fp:  
  ball_pos = pickle.load(fp)
print(ball_pos)
last_f, last_pos = 0,[]
for f, pos in ball_pos.items():
  if f-last_f < 10 and last_pos != []:
    vel = [pos[0][0] - last_pos[0][0], pos[0][1] - last_pos[0][1]]
    vel = [v/ (f-last_f) for v in vel]
    pos[0].append(vel)
  last_f, last_pos = f, pos

{12: [[5, 389]], 26: [[521, 193]], 29: [[521, 213]], 30: [[522, 224]], 31: [[523, 235]], 32: [[520, 246]], 33: [[520, 246]], 34: [[522, 238]], 35: [[522, 234]], 36: [[522, 228]], 37: [[520, 222]], 38: [[521, 218]], 39: [[522, 220]], 41: [[521, 211]], 50: [[598, 228]], 52: [[619, 237]], 53: [[636, 244]], 54: [[656, 252]], 55: [[676, 264]], 56: [[698, 276]], 57: [[694, 273]], 58: [[716, 286]], 61: [[780, 330]], 62: [[804, 346]], 63: [[802, 348]], 64: [[828, 366]], 65: [[848, 384]], 66: [[874, 406]], 67: [[900, 430]], 68: [[920, 450]], 71: [[1119, 488]], 72: [[1114, 490]], 73: [[1108, 492]], 78: [[904, 390]], 79: [[880, 370]], 80: [[854, 348]], 81: [[855, 350]], 82: [[834, 334]], 83: [[814, 320]], 84: [[797, 308]], 86: [[759, 286]], 87: [[760, 288]], 88: [[742, 278]], 89: [[723, 271]], 90: [[710, 266]], 91: [[694, 262]], 92: [[680, 258]], 93: [[681, 256]], 95: [[662, 262]], 96: [[660, 268]], 97: [[656, 276]], 98: [[652, 284]], 100: [[649, 294]], 104: [[632, 340]], 105: [[632, 340]], 107: 

#Analisis

In [8]:
print(len(person_results))
print(len(ball_pos))

300
203


In [9]:
import numpy as np
import cv2

def plot_court(img):
  vis_img = cv2.polylines(img, np.int32([pista_campo_fondo]), True, (255,150,255),3)
  vis_img = cv2.polylines(img, np.int32([pista_campo_cerca]), True, (255,70,255),3)
  return vis_img

image = cv2.imread('/content/gdrive/MyDrive/Colab Notebooks/mmpose/last_img.png')
cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/mmpose/last_img_court.png', plot_court(image))

True

In [10]:
import numpy as np
import matplotlib.path as mpltPath
import math
import random
def rectangle_dist(bbox1, bbox2):
  if len(bbox1)>=4 and len(bbox2)>=4:
    c1 = [(bbox1[0]+bbox1[2])/2, (bbox1[1]+bbox1[3])/2]
    c2 = [(bbox2[0]+bbox2[2])/2, (bbox2[1]+bbox2[3])/2]
    d =  math.sqrt( (bbox2[0] - bbox1[0])**2 + (bbox2[1] - bbox1[1])**2 )
    d += math.sqrt( (bbox2[0] - bbox1[0])**2 + (bbox2[3] - bbox1[3])**2 )
    d += math.sqrt( (bbox2[2] - bbox1[2])**2 + (bbox2[1] - bbox1[1])**2 )
    d += math.sqrt( (bbox2[2] - bbox1[2])**2 + (bbox2[3] - bbox1[3])**2 )
    if d == 0:
      return d+random.random()/100
    return d/4
  elif len(bbox1)>=2 and len(bbox2)>=2:
    c1, c2 = bbox1, bbox2
    return math.sqrt( (c2[0] - c1[0])**2 + (c2[1] - c1[1])**2 )
  else:
    return None

pista = mpltPath.Path(pista_aristas)
#pista_c0 = mpltPath.Path(pista_campo_cerca)

def persons_in_court(persons, pista):
  players=[]
  for i in range(len(persons)):
    p = persons[i]
    if pista.contains_points([[p['bbox'][2],p['bbox'][3]]]) or \
        pista.contains_points([[p['bbox'][0],p['bbox'][3]]]):
      players.append(i)
      if len(players) == 4:
        break
  return [persons[i]['bbox'] for i in range(len(persons)) if i in players], \
          [persons[i]['bbox'] for i in range(len(persons)) if i not in players]

#players_box = []
#for f in person_results:
#  players_box.append(persons_in_court(f, pista))
#print(players_box)
  #print(p['bbox'])

#cv2.putText(img, str(i), (p['bbox'][0],p['bbox'][3]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (10, 255, 0), 1, cv2.LINE_AA)
#players_box
#cv2.imwrite('/content/gdrive/MyDrive/Colab Notebooks/mmpose/last_img_court.png', img)
#path.contains_points([(100,100)])

## Track jugador cercano

In [11]:

import copy
def identify_players(persons, last_players, pista, debug = False):
  distances = []
  if len(persons) == 0:
    return []
  
  p_in_court, p_not_in_court = persons_in_court(persons, pista)
  if debug: print("--->debug call identify_players")
  if debug: print(len(last_players))
  if len(last_players) == 0:
    return [np.append(p_in_court[i], int(i)) for i in range(len(p_in_court))]
  
  players = []
  
  if len(p_in_court) > 0: #Asignar los jugadores dentro de la pista con su posición previa más cercana
    candidates = p_in_court
    distances_m = np.array([[rectangle_dist(p, person) for person in candidates] for p in last_players])
    while distances_m.size >0 and len(players) < 4 and distances_m.min() < 30:
      if debug: print(distances_m)
      distances_min = distances_m.min()
      if debug: print(distances_min)
      ij_min = np.where(distances_m == distances_min)
      
      (i_min,j_min) = tuple([i.item() for i in ij_min])

      pl = candidates[j_min] 
      pl = np.append(pl, last_players[i_min][5])
      pl = np.append(pl, distances_min)
      players.append(pl)
      if debug: print("player "+str(pl[5])+" localizado dentro de pista")
      if debug:print(last_players[i_min])
      if debug:print(candidates[j_min])
      #if debug:print((i_min,j_min))
      #distances_m = np.delete(distances_m, i_min, 0)
      #distances_m = np.delete(distances_m, j_min, 1)
      del last_players[i_min]
      del candidates[j_min]
      distances_m = np.array([[rectangle_dist(p, person) for person in candidates] for p in last_players])
    if len(last_players) == 0:
      while len(p_in_court) > 0 and len(players) < 4: #nuevos jugadores
        new_id = [i for i in range(4) if i not in [p[5] for p in players]][0]
        pl = np.append(p_in_court[0], new_id)
        pl = np.append(pl, -1)
        players.append(pl)
        if debug: print("nuevo player "+str(pl[5])+" detectado")
        del p_in_court[0]
  
  candidates = p_not_in_court
  if len(candidates) > 0 and len(last_players) > 0 and len(players) < 4: #buscar jugadores fuera de pista
    distances_m = np.array([[rectangle_dist(p, person) for person in candidates] for p in last_players])
    while distances_m.size >0 and len(players) < 4 and distances_m.min() < 30:
      distances_min = distances_m.min()
      ij_min = np.where(distances_m == distances_min)
      
      #print(distances_min)
      i_min, j_min = tuple([i.item() for i in ij_min])

      pl = candidates[j_min] 
      pl = np.append(pl, last_players[i_min][5])
      pl = np.append(pl, distances_min)
      players.append(pl)
      if debug: print("player "+str(pl[5])+" localizado dentro de pista")
      if debug:print(last_players[i_min])
      if debug:print(candidates[j_min])
      #if debug:print((i_min,j_min))
      #distances_m = np.delete(distances_m, i_min, 0)
      #distances_m = np.delete(distances_m, j_min, 1)
      del last_players[i_min]
      del candidates[j_min]
      distances_m = np.array([[rectangle_dist(p, person) for person in candidates] for p in last_players])
    
  return players


players_box = [[]]
for i in range(len(person_results)):
  players_box.append(identify_players(copy.deepcopy(person_results[i]), copy.deepcopy(players_box[-1]) if len(players_box)>0 else [], pista, False))
#  print("players frame "+str(i)+":")
#  print(players_box[i])
#print(players_box[1351])
#print(players_box[1352])
#print(max([max([p[-1] for p in per]) for per in players_box if per != []]))

### Procesado de pista y detector de plano central

In [22]:
import math
import numpy as np
import pandas as pd
import cv2
from skimage import io
from PIL import Image 
import matplotlib.pylab as plt
from keras.preprocessing import image

t_points = [[637, 251], [637, 518], 25]

def avg(lst):
    return sum(lst)/len(lst)

def group_similar_lines(lines, v_h = 0):
  res_lines = []
  for l in lines:
    dist = [[abs((l[0+v_h]+l[2+v_h])/2 - avg([(s[0+v_h]+s[2+v_h])/2 for s in res_lines[i]])) , i] for i in range(len(res_lines))]
    if dist != []:
      dist.sort(key=lambda a: a[0])
      if dist[0][0] < 5:
        res_lines[dist[0][1]].append(l)
      else:
        res_lines.append([l])
    else:
      res_lines.append([l])
  res = []
  for g in res_lines:
    x = int(round(avg([(l[0+v_h]+l[2+v_h])/2 for l in g]), 0))
    width = max([max(l[0+v_h],l[2+v_h]) for l in g]) - min([min(l[0+v_h],l[2+v_h]) for l in g])   
    y_min = int(round(min([min([l[1-v_h],l[3-v_h]]) for l in g]), 0))
    y_max = int(round(max([max([l[1-v_h],l[3-v_h]]) for l in g]), 0))
    if v_h==0: #vertical line
      res.append([x, y_min, x, y_max, width])
    else: #horizontal line
      res.append([y_min, x, y_max, x, width])
  return res

def line_intersection_vh(line_v, line_h):
    if line_v[0] <= max(line_h[0], line_h[2]) and \
        line_v[0] >= min(line_h[0], line_h[2]) and \
        line_h[1] <= max(line_v[1], line_v[3]) and \
        line_h[1] >= min(line_v[1], line_v[3]):
      return [line_v[0], line_h[1]]
    return None

def is_central_camera(img):
  dst = cv2.Canny(img, 50, 200, None, 3)
   
  # Copy edges to the images that will display the results in BGR
  #cdst = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
  #cdstP = np.copy(cdst)
  #cv2_imshow( cdstP)
    
  #lines = cv2.HoughLines(dst, 1, np.pi / 180, 150, None, 0, 0)
    
  #if lines is not None:
  #  for i in range(0, len(lines)):
  #    rho = lines[i][0][0]
  #    theta = lines[i][0][1]
  #    a = math.cos(theta)
  #    b = math.sin(theta)
  #    x0 = a * rho
  #    y0 = b * rho
  #    pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
  #    pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
  #    cv.line(cdst, pt1, pt2, (0,255,0), 3, cv.LINE_AA)

  linesP = cv2.HoughLinesP(dst, 1, np.pi / 180, 50, None, 50, 10)
  instersection = []
  if linesP is not None:
    h_lines = [l[0] for l in linesP if abs(l[0][1]-l[0][3])<3]
    v_lines = [l[0] for l in linesP if abs(l[0][0]-l[0][2])<3]
    
    v_lines = group_similar_lines(v_lines)
    h_lines = group_similar_lines(h_lines, 1)
    #print(v_lines)
    for v in v_lines:
      for h in h_lines:
        instersection.append(line_intersection_vh(v, h))
    instersection = [i for i in instersection if i]
    if instersection != []:
      t_point_detected = 0
      for p in instersection:
        img = cv2.circle(img, (int(p[0]), int(p[1])), 8, (0,0,255), -1)
        if rectangle_dist(p, t_points[0]) < 25 or \
            rectangle_dist(p, t_points[1]) < 25:
            t_point_detected +=1
      #print(t_point_detected)
      if t_point_detected >= 2:
        return img, True
      
  return img, False
    


# Imprimir resultado a video

In [21]:
import cv2
def plot_players(img, players):
  i = 1
  for p in players:
    img = cv2.rectangle(img, (int(p[0]), int(p[1])), (int(p[2]), int(p[3])), (255,0,0), 2)
    cv2.putText(img, str(p[5]), (int(p[2]),int(p[3])), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (10, 255, 0), 1, cv2.LINE_AA)
    cv2.putText(img, str(round(p[-1],2)), (int((p[2]+p[0]-6)/2),int(p[1]-4)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (10, 255, 0), 1, cv2.LINE_AA)
    i += 1
  return img

def plot_ball(img, ball):
  center = (int(ball[0]), int(ball[1]))
  img = cv2.circle(img, center, 5, (0,0,255), 2)
  if len(ball)>2:
    speed = (int(ball[0]+1.5*ball[2][0]), int(ball[1]+1.5*ball[2][1]))
    img = cv2.arrowedLine(img, center, speed, (0,0,200), 2)
  return img

#players_box = [[p['bbox'] for p in per] for per in person_results]

video_path = path+input_name+'.mp4'
out_video_root = path+'vis_'+input_name+'.mp4'
print(out_video_root)
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter(out_video_root, fourcc, fps, size)
i = 0
while (cap.isOpened()):
        flag, img = cap.read()
        if not flag:
            break
        #modify img
        vis_img, camera_flag = is_central_camera(img)
        if camera_flag:
          cv2.putText(vis_img, str(i), (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (10, 255, 0), 1, cv2.LINE_AA)
        else:
          cv2.putText(vis_img, str(i), (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (153, 0, 0), 1, cv2.LINE_AA)
        vis_img = plot_court(vis_img)
        
        if i < len(players_box):
          vis_img = plot_players(vis_img, players_box[i])
        if i in ball_pos:
          vis_img = plot_ball(vis_img, ball_pos[i][0])

        #save img
        videoWriter.write(vis_img)
        i += 1
cap.release()
videoWriter.release()

/content/gdrive/MyDrive/PadelData/vis_punto_qatar.mp4
0
0
0
0
0
0
0
0
6
0
3
1
3
3
1
3
0
3
0
0
1
2
2
2
4
0
0
0
3
0
0
1
4
0
4
4
1
3
0
0
2
0
1
2
2
3
3
3
1
4
3
2
0
4
0
5
4
4
4
0
2
0
5
4
2
0
3
6
0
6
0
4
0
1
6
0
4
0
1
0
0
0
1
6
0
0
1
6
0
1
0
1
4
1
6
1
0
1
0
1
5
0
3
4
5
1
0
4
2
1
0
0
1
4
5
5
3
0
3
4
4
4
5
2
1
4
3
1
4
7
0
2
2
4
1
5
3
3
1
4
5
1
0
0
3
0
1
1
3
0
0
0
2
0
0
4
1
0
2
5
2
0
0
2
3
4
5
1
3
2
0
5
0
3
3
0
2
1
2
9
1
2
5
1
5
5
5
2
2
2
4
5
0
1
1
4
3
3
3
6
0
5
1
2
2
1
4
1
0
4
5
2
7
3
6
2
9
5
4
3
5
4
4
1
2
5
0
2
3
0
5
2
5
1
7
1
3
4
8
0
1
3
3
1
1
1
4
2
3
3
2
5
1
4
4
1
1
0
2
1
4
1
1
1
2
4
1
4
3
1
4
1
3
2
1
5
1
4
4
1
1
0
3
4
1
1
1
1
3
1
5
1
3
2
4
